<div style="text-align:center">
    <img src="https://codeflex.co/wp-content/uploads/2021/03/python3-multiprocessing-example.jpg" />
</div>



# Multiprocessing en Python: Paralelismo basado en procesos #

### Introducción
En Python, el módulo multiprocessing permite crear procesos independientes para realizar tareas concurrentemente, aprovechando así múltiples núcleos de CPU y logrando paralelismo. A diferencia del módulo threading, multiprocessing usa procesos en lugar de subprocesos, lo que evita los problemas de concurrencia asociados con el Global Interpreter Lock (GIL) de Python.

### Funcionamiento
El módulo multiprocessing proporciona una API similar a la del módulo threading, pero utiliza procesos en lugar de subprocesos. Cada proceso tiene su propio espacio de memoria, lo que evita problemas de concurrencia. Los procesos se pueden crear y manejar fácilmente utilizando clases como Process, Pool y Queue.

### Casos de uso
Procesamiento por lotes: Procesar grandes cantidades de datos de manera eficiente mediante la distribución de la carga de trabajo en múltiples procesos.
Cálculos intensivos: Realizar cálculos complejos distribuyendo la carga entre varios procesos para mejorar el rendimiento.
Paralelización de tareas de E/S: Mejorar la eficiencia al realizar operaciones de entrada/salida, como lectura/escritura de archivos o solicitudes de red, en paralelo.
### Ejemplo básico
Supongamos que queremos calcular la suma de una lista grande de números de forma paralela utilizando multiprocessing.

In [1]:
import multiprocessing

def calcular_suma(lista):
    return sum(lista)

if __name__ == "__main__":
    lista_grande = list(range(1000000))
    num_procesos = multiprocessing.cpu_count()
    
    chunk_size = len(lista_grande) // num_procesos
    chunks = [lista_grande[i:i+chunk_size] for i in range(0, len(lista_grande), chunk_size)]
    
    with multiprocessing.Pool(processes=num_procesos) as pool:
        resultados = pool.map(calcular_suma, chunks)
    
    suma_total = sum(resultados)
    print("La suma total es:", suma_total)


`La suma total es: 499999500000`
### En este ejemplo
dividimos la lista grande en partes más pequeñas (chunks) y luego utilizamos un grupo de procesos (Pool) para calcular la suma de cada parte en paralelo. Finalmente, sumamos los resultados parciales para obtener la suma total.

### Uso más común
El uso más común de multiprocessing es la aceleración de cálculos intensivos y tareas de procesamiento de datos en paralelo. Por ejemplo, en aplicaciones de análisis de datos, procesamiento de imágenes, cálculos científicos, etc.

In [ ]:
import multiprocessing
from PIL import Image, ImageFilter

def aplicar_filtro(imagen, filtro, resultado, indice):
    imagen_filtrada = imagen.filter(filtro)
    resultado.put((indice, imagen_filtrada))

if __name__ == "__main__":
    # Abrir la imagen
    imagen_original = Image.open("imagen.jpg")
    
    # Definir el filtro a aplicar
    filtro = ImageFilter.BLUR
    
    # Dividir la imagen en partes iguales para cada proceso
    ancho, alto = imagen_original.size
    num_procesos = multiprocessing.cpu_count()
    tamano_subimagen = alto // num_procesos
    subimagenes = [(0, i * tamano_subimagen, ancho, (i + 1) * tamano_subimagen) for i in range(num_procesos)]
    
    # Crear un objeto Queue para recopilar los resultados de los procesos
    resultados = multiprocessing.Queue()
    
    # Crear y ejecutar los procesos
    procesos = []
    for i, (x0, y0, x1, y1) in enumerate(subimagenes):
        subimagen = imagen_original.crop((x0, y0, x1, y1))
        p = multiprocessing.Process(target=aplicar_filtro, args=(subimagen, filtro, resultados, i))
        procesos.append(p)
        p.start()
    
    # Esperar a que todos los procesos terminen
    for p in procesos:
        p.join()
    
    # Ordenar los resultados y reconstruir la imagen filtrada
    resultados_ordenados = [resultados.get() for _ in range(num_procesos)]
    resultados_ordenados.sort()
    subimagenes_filtradas = [imagen for indice, imagen in resultados_ordenados]
    imagen_filtrada = Image.new("RGB", (ancho, alto))
    y = 0
    for subimagen_filtrada in subimagenes_filtradas:
        imagen_filtrada.paste(subimagen_filtrada, (0, y))
        y += subimagen_filtrada.size[1]
    
    # Guardar la imagen filtrada
    imagen_filtrada.save("imagen_filtrada.jpg")


### Process
Process es una clase que representa un proceso independiente en Python. Se utiliza para crear nuevos procesos mediante la definición de una función objetivo que será ejecutada en el proceso.

Métodos importantes:
1. __init__(self, target, args=(), kwargs={}): Constructor de la clase Process. Toma la función target como argumento, así como los argumentos posicionales args y los argumentos de palabras clave kwargs que se pasarán a la función.
2. start(): Inicia la ejecución del proceso.
3. join(timeout=None): Espera a que el proceso termine. Opcionalmente, se puede especificar un tiempo de espera (timeout) en segundos.
### Ejemplo:

In [1]:
import multiprocessing

def worker(num):
    print(f"Proceso hijo: {num}")

if __name__ == "__main__":
    processes = []
    for i in range(5):
        p = multiprocessing.Process(target=worker, args=(i,))
        processes.append(p)
        p.start()
    
    for p in processes:
        p.join()


`Proceso hijo: 0
Proceso hijo: 1
Proceso hijo: 2
Proceso hijo: 3
Proceso hijo: 4`

### Process (Continuación)

Métodos importantes (Continuación):
1. is_alive(): Devuelve True si el proceso está vivo (ejecutándose), False de lo contrario.
2. terminate(): Termina el proceso de forma abrupta.
3. pid: Atributo que devuelve el identificador de proceso (PID) del proceso.

### Ejemplo:

In [2]:
import multiprocessing
import time

def mi_tarea():
    print("Comenzando la tarea...")
    time.sleep(2)
    print("Terminando la tarea...")

if __name__ == "__main__":
    proceso = multiprocessing.Process(target=mi_tarea)
    print("Estado del proceso antes de iniciar:", proceso.is_alive())
    
    proceso.start()
    print("Estado del proceso después de iniciar:", proceso.is_alive())
    
    proceso.join()
    print("Estado del proceso después de terminar:", proceso.is_alive())


Estado del proceso antes de iniciar: False
Estado del proceso después de iniciar: True
Estado del proceso después de terminar: False


En este ejemplo, creamos un proceso que ejecuta una tarea (mi_tarea). Después de crear el proceso, podemos verificar su estado utilizando el método is_alive(). Luego, iniciamos el proceso, verificamos su estado nuevamente y finalmente esperamos a que termine la tarea utilizando join().

### También podemos obtener el PID del proceso usando el atributo pid:

In [3]:
print("PID del proceso:", proceso.pid)


PID del proceso: 11960


### uso de if __name__ == "__main__":
Es importante envolver el código que crea procesos dentro de la condición if __name__ == "__main__": para evitar problemas con la importación del módulo. Cuando se importa un módulo que contiene código para crear procesos, el intérprete de Python intentará crear esos procesos nuevamente en el proceso principal, lo que puede provocar comportamientos inesperados.

Al colocar el código de creación de procesos dentro de esta condición, nos aseguramos de que solo se ejecute cuando el módulo se ejecute como un script independiente y no cuando se importe como un módulo en otro script.



### Conclusión
El módulo multiprocessing de Python proporciona una forma efectiva de aprovechar el paralelismo basado en procesos para mejorar el rendimiento de las aplicaciones. Al distribuir tareas en múltiples procesos, podemos aprovechar eficientemente los recursos del sistema y realizar operaciones de manera concurrente. Es importante tener en cuenta el costo asociado con la creación y gestión de procesos, por lo que su uso debe evaluarse cuidadosamente según las necesidades específicas de cada aplicación.

